# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo implementar um abiente seguro, constituído por um emissor e um recetor, através de sessões cifradas e seguras.
A solução implmentada, foi estruturada da seguinte forma:

- Construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- Utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes, na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Implementar novamente o cenário anterior mas, desta feita vez, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Sessão Síncrona com _Diffie-Hellman_

### Imports

De seguida, encontram-se os módulos **Python** importados e necessários para desenvolver a sessão síncrona entre os agentes, utilizando o protocolo **DH** e o **DSA**.

In [1]:
import os,io
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import *
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição de funções auxiliares 

Esta secção define funções auxiliares tais como, a função que gera o **hash** de uma mensagem, de **cifragem** e de **decifragem**, que serão posteriormente utilizados pelos agentes que participam na sessão.

In [2]:
def Hash(s):
    digest = hashes.Hash(hashes.SHA256(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

def next_tweak(initial, current_counter):
    str = format(current_counter, '063b')
    str = str + '0'
    b_str = bytes(str,'utf-8')
    return (initial + b_str)

# Algo está errado numa destas operações!
# A cifragem e decifragem não está a dar o mesmo resultado!
# Função bytes_xor não a conheces. poderá ser daí?
# Ou a operação de decifra deve ter algo de diferente em relação à operação de cifra?

def bytes_xor(a, b) :
    return bytes(x ^ y for x, y in zip(a, b))

def cifra(key,block,tweak,cipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,tweak)
    e_s_x_hw = cipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram

def decifra(key,block,tweak,decipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,tweak)
    e_s_x_hw = decipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram


# Falta pôr funções de cifra e de decifra
    

### Definição da classe de multiprocessamento

Embaixo definem-se a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [3]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def auto(self,proc=None):
        if proc == None:
            self.left_process.start()
            self.right_process.start()
            self.left_process.join(self.timeout)
            self.right_process.join(self.timeout)
        else:
            proc.start()
            proc.join()
    
    def manual(self): 
        self.left()
        self.right()

### Geração dos parâmetros _Diffie-Hellman_

A próxima secção implementa a geração dos parâmetros necessários para a derivação de chaves **DH** e **DSA**, por parte de ambos os agentes.

In [4]:
parameters_dh = dh.generate_parameters(generator=2,key_size=1024,backend=default_backend())
# O tamanho tem que ser 3072 bits, alterar depois.

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes dos agentes envolvidos na comunicação, tendo em conta que estas são conhecidas por ambos, e serão aplicadas na assinatura digital das mensagens.

In [5]:
# Geração das chaves do Emissor
emitter_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
emitter_dsa_pk = emitter_dsa_sk.public_key() #chave pública DSA

# Geração das chaves do Recetor
receiver_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
receiver_dsa_pk = receiver_dsa_sk.public_key() #chave pública DSA

### Definição do Emissor e do Recetor

Por fim definiu-se o comportamento de cada um dos agentes nesta sessão síncrona segura de troca de informação. Na definição de ambos, é assumido que as chaves públicas **DSA** utilizadas na autenticação dos agentes já são conhecidas por ambos. As chaves **DH** são geradas por sessão de modo a acordar uma chave temporária, para que seja possível cifrar e decifrar uma mensagem utilizando uma primitiva simétrica.

**PROCESSO DE CIFRA E DECIFRA NÃO ESTÁ CORRECTO! NÃO PRODUZ OS MESMOS RESULTADOS**

In [6]:
def Emitter(connection):
    #Implementação Protocolo Diffie-Hellman
    emitter_dh_sk = parameters_dh.generate_private_key()
    emitter_dh_pk = emitter_dh_sk.public_key()
    pub = emitter_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = emitter_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk': pub, 'sig': signature}
    connection.send(message)
    receiver_first = connection.recv()
    receiver_dh_public_key = serialization.load_pem_public_key(receiver_first['pk'],default_backend())
    receiver_dsa_pk.verify(receiver_first['sig'],receiver_first['pk'],hashes.SHA256())
    master_key = emitter_dh_sk.exchange(receiver_dh_public_key)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    message_size = 2**10
    gen_bytes = os.urandom(message_size)
    inputs = io.BytesIO(gen_bytes)
    #inicialização do tweak como um nounce de 64 bits (n/2)
    tweak_init = os.urandom(8)
    
    connection.send(tweak_init)
    
    buffer = bytearray(16)
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).encryptor()
    i = 1
    try:
        while inputs.readinto(buffer):
            # Falta ver a parte de autenticação dos superblocos do criptograma
            current_tweak = next_tweak(tweak_init,i)
            cryptogram = cifra(shared_key,bytes(buffer),current_tweak,cipher)
            connection.send(cryptogram)
            i += 1
        connection.send(cipher.finalize())
    except Exception:
        print('Erro no emissor!')
    connection.send('finalized')   
    inputs.close()
    connection.close()    

In [7]:
def Receiver(connection):
    #Implementação protocolo Diffie-Hellman
    emitter_first = connection.recv()
    receiver_dh_sk = parameters_dh.generate_private_key()
    receiver_dh_pk = receiver_dh_sk.public_key()
    emitter_dh_public_key = serialization.load_pem_public_key(emitter_first['pk'],default_backend())
    emitter_dsa_pk.verify(emitter_first['sig'],emitter_first['pk'],hashes.SHA256())
    master_key = receiver_dh_sk.exchange(emitter_dh_public_key)
    pub = receiver_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = receiver_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk':pub,'sig':signature}
    connection.send(message)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    outputs = io.BytesIO()
    tweak_init = connection.recv()
    
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).decryptor()
    i = 1
    try:
        while True:
            buffer = connection.recv()
            if(buffer == 'finalized'):
                outputs.write(cipher.finalize())
                break
            else:
                #print(buffer)
                cryptogram = buffer
                current_tweak = next_tweak(tweak_init,i)
                text = decifra(shared_key,cryptogram,current_tweak,cipher)
                i += 1
                outputs.write(text)
        print(outputs.getvalue())
    except Exception as Err:
        print('Erro no recetor! {0}'.format(Err))
        
    outputs.close()
    connection.close()
    

In [8]:
BiConnection(Emitter,Receiver).auto()

b'p\xcb\xfa\x16gCX-\xd0\x7f\x7f\xff\xef\xc2\x91\x8cM\x95\xd4\x96\xb1\xc0p\x13h\xf0\x16\xbb\xa0\nw9,oV\x12\xfd\xb0{;\x12\x14\xdaW$\x87\xe0\xfd\x96\x8a\x96\x85\xdfSU\xe6%\x82h3\x0c}\x00+\x1b`!E\x13\xae\xb0:\xb1yj\x97N+/\xce>\xd0\xd1\xef\xd3/\x99/\xfc\xdaCF\x19\xf1`\x90"\xc770\xd1\x99\x1e\xe7\x9dd]\xaa\x05\xd66"\x8eJ\xef5*s\x8f\x02\xb2y\xcd\xa3\xc8\xf4\xbdv\xfe\xbaLq1\x1c\xe5*\xd3\xab\xe3a\x90EE-\x0e\xf9<\xc2\xe1[\x00.5\xcb\xb3k\x89\xffF\x84\xcc\xc0\xa0\xcc\x08\xe3a9f;\xa5L\xc5\xc6Z\x15\x86\x15\x1a\xa5\xb6\x05Z\x9e\xd4]E/c!\x00\x8fmm\x05 \x8b>C\xc7\xa3\'\xdau\xe0\xb1\x92\x84Z\x96\x9boD\xf3\xcaWSn\xfa\x81\xbc]\xa6G\x8ba\x1cU\xa9\tZ\xc3\xf2gf\x12\x85n\x0cQ\n\x1aW\xfc\x80\x14T\xa5\xc9\xd6\x04P\xdfS\x864H\x1d\x02C\xdf?\xb5\xa1Y\xa8/\xce\xde\x83z\xb1\x1c\xebD\x04<\x19\xcd6\xc4\x04\xdaC\x18\xbf\xc3X,\xe8\xc9\xfa\xcfg\xf8\xed\x05\xb8\x84&\xd7\xda\xc5}\xd3\x8e\x99\x9fJ~k \xb3\xd2Q\xe8\x8c\x17CQ\xdf\x94Nb\x196O \xf0\xc7\x997\xb1\xd1U!i)\x14++.e\xbd\xa0\x80\xc5z\x89\xe3\xde\xc3\xffT\xf8\xe4\xd19C))